# attend-dash
 Currently this notebook is divided into parts. Run only the parts you need. Run the first part the first time you open this notebook.

## 1. Scraping

In [5]:
import requests

In [6]:
url = "http://apf.ststephens.edu/attendance_stku2018"

payload = {
'programme':'33',
'semester':'10',
'month':'07'
}

In [7]:
r = requests.post(url, data=payload)


In [8]:
f = open('response.html', 'w')
f.write(r.text)

631013

## 2. Cleaning

In [2]:
from bs4 import BeautifulSoup


In [10]:
pwd

'C:\\Users\\abhis\\Desktop\\test_scrape\\attend-dash'

In [3]:
f = open('response.html', 'r') 

soup = BeautifulSoup(f.read(),'html.parser')

In [26]:
def is_paper_name(colspan):
    ''' Paper names have headers which are 6 columns wide '''
    return colspan and colspan == '6'

headers = {paper_name.text.strip() for paper_name  in soup.find_all(colspan = is_paper_name)}
headers

{'Computer Networks and Internet Technologies',
 'Group Theory - I',
 'Indian Economy - I',
 'Latex and HTML SEC Group I',
 'Latex and HTML SEC Group II',
 'Making of Contemporary India',
 'Media and  Communications GE II',
 'Money and Banking',
 'Multivariate Calculus',
 'Multivariate Calculus Group I',
 'Multivariate Calculus Group II',
 'Theory of Functions'}

In [6]:
def is_student_record(class_attr):
    return class_attr and class_attr == 'trhover'
data = soup.find_all('tr',class_='trhover')
attendance = data[0].get_text().splitlines()

In [7]:
attendance

['',
 'Adya  Singh',
 '5',
 '0',
 '0',
 '0',
 '0',
 '0',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '8',
 '7',
 '0',
 '0',
 '0',
 '0',
 '9',
 '8',
 '0',
 '0',
 '0',
 '0',
 '5',
 '5',
 '0',
 '0',
 '0',
 '0',
 '',
 '',
 '',
 '',
 '',
 '',
 '0',
 '0',
 '0',
 '0',
 '4',
 '2',
 '',
 '',
 '',
 '',
 '',
 '',
 '3',
 '3',
 '0',
 '0',
 '5',
 '5',
 '30',
 '23',
 '76.67',
 '0',
 '0',
 '0.00',
 '9',
 '7',
 '77.78                                                        ',
 '76.67',
 '77.78']

In [18]:
attendance_list = []
for index,subject in enumerate(headers):
    attendance_list.append([subject,attendance[2+index*6:2+(index+1)*6]])
attendance_list

[['Media and  Communications GE II', ['5', '0', '0', '0', '0', '0']],
 ['Making of Contemporary India', ['', '', '', '', '', '']],
 ['Indian Economy - I', ['', '', '', '', '', '']],
 ['Money and Banking', ['', '', '', '', '', '']],
 ['Computer Networks and Internet Technologies', ['', '', '', '', '', '']],
 ['Theory of Functions', ['8', '7', '0', '0', '0', '0']],
 ['Group Theory - I', ['9', '8', '0', '0', '0', '0']],
 ['Multivariate Calculus', ['5', '5', '0', '0', '0', '0']],
 ['Multivariate Calculus Group I', ['', '', '', '', '', '']],
 ['Multivariate Calculus Group II', ['0', '0', '0', '0', '4', '2']],
 ['Latex and HTML SEC Group I', ['', '', '', '', '', '']],
 ['Latex and HTML SEC Group II', ['3', '3', '0', '0', '5', '5']]]

In [42]:
headers

{'Computer Networks and Internet Technologies',
 'Group Theory - I',
 'Indian Economy - I',
 'Latex and HTML SEC Group I',
 'Latex and HTML SEC Group II',
 'Making of Contemporary India',
 'Media and  Communications GE II',
 'Money and Banking',
 'Multivariate Calculus',
 'Multivariate Calculus Group I',
 'Multivariate Calculus Group II',
 'Theory of Functions'}

In [48]:
list(headers)[4].split()[-2]

'Group'

In [54]:
list(headers)

['Latex and HTML SEC Group II',
 'Latex and HTML SEC Group I',
 'Group Theory - I',
 'Making of Contemporary India',
 'Multivariate Calculus Group II',
 'Money and Banking',
 'Indian Economy - I',
 'Multivariate Calculus Group I',
 'Theory of Functions',
 'Media and  Communications GE II',
 'Multivariate Calculus',
 'Computer Networks and Internet Technologies']

In [60]:
# Paper names must not contain group as the second last word
papers = { header_name for header_name in headers if header_name.split()[-2] != 'Group' }
tuts = headers - papers
papers

{'Computer Networks and Internet Technologies',
 'Group Theory - I',
 'Indian Economy - I',
 'Making of Contemporary India',
 'Media and  Communications GE II',
 'Money and Banking',
 'Multivariate Calculus',
 'Theory of Functions'}

In [61]:
tuts

{'Latex and HTML SEC Group I',
 'Latex and HTML SEC Group II',
 'Multivariate Calculus Group I',
 'Multivariate Calculus Group II'}

### TODO
1. Scrape individual attendance data per student 
2. Calculate necessary values for aggregate calculations and store them in dataframe or np.array
3. Prepare data to display for individual summary and aggregate summary

### Potential individual summary format
- name
- number of papers
- for each paper
    - paper name
    - paper attendance
- total attendance